<a href="https://colab.research.google.com/github/dikisp/PWA-/blob/master/tanahairku_with_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import csv
from collections import defaultdict
import re
import json



from scipy.stats import pearsonr

from matplotlib.colors import ListedColormap
from scipy import stats

from wordcloud import WordCloud


from sklearn.cluster import KMeans

from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install firebase_admin

In [ ]:
import firebase_admin
from firebase_admin import credentials, db

In [ ]:
cred = credentials.Certificate("sdk-firebase.json")
firebase_admin.initialize_app(cred, {
    'databaseURL' : 'https://tanahairku-experimental-cl.firebaseio.com/'
})

In [ ]:
ref = firebase_admin.db.reference("/users")

data = ref.get()
log_data = []
for userid in data:
   for id_log in data[userid]['log'] :
     log_data.append([
       data[userid]['log'][id_log]['Date'],
       id_log,
       data[userid]['log'][id_log]['Items_Interval'],
       data[userid]['log'][id_log]['Items_Log'],
     ])

# log_data


In [ ]:
df=pd.DataFrame(log_data)

In [ ]:
new_names = {0 : 'tgl_akses',
             1 : 'id_user',
             2 : 'durasiAkses',
             3 : 'konten'
            }
df.rename(columns=new_names, inplace=True)

In [ ]:
# # #mengisi yang nan
df = df[pd.notnull(df['durasiAkses'])]

In [ ]:
# #drop 0
drop_0=df.index[df["durasiAkses"] == '0'].tolist()
df=df.drop(df.index[drop_0])

In [ ]:
df_konten_list = df['konten'].tolist()

In [ ]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""    
    # traverse in the string   
    for ele in s:  
        str1 += ele       
    # return string   
    return str1         
# Driver code     
s = df_konten_list
result_list_konten = listToString(s)
res_konten = result_list_konten.split(',')

In [ ]:
#hilaingin start
string = result_list_konten
hasil_stirng = string.replace('START,',',')
del_start = hasil_stirng.split(',')

In [ ]:
df_interval_list = df['durasiAkses'].tolist()
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""    
    # traverse in the string   
    for ele in s:  
        str1 += ele       
    # return string   
    return str1         
# Driver code     
s = df_interval_list
result_list_interval = listToString(s)


#str split
res_interval = result_list_interval.split(',')


In [ ]:
#regex for del end char
# for i in result_list_interval:
#     len_char = len(i)
#     print(len_char)  

In [ ]:
#list to dataframe konten
df_res_konten = pd.DataFrame(del_start)

#list to dataframe interval
df_res_interval = pd.DataFrame(res_interval)
df_res_interval

#tglakses
tglakses = df.tgl_akses
#user_id
user_id = df.id_user

# concat keduanya
result_finish = pd.concat([tglakses,user_id,df_res_konten, df_res_interval], axis=1, sort=False)
df = result_finish

In [ ]:
df

,tgl_akses,id_user,0,0
0,NaN,NaN,,0
1,13-02-2020,-M-wZQJsrdMbk7RBbm8y,Pakaian_Asmat,47
2,NaN,NaN,Rumah_Asmat,30
3,NaN,NaN,Senjata_Asmat,1
4,NaN,NaN,Senjata_Asmat,28
...,...,...,...,...
490,NaN,NaN,Senjata_Asmat,116
491,NaN,NaN,Senjata_Asmat,4
492,NaN,NaN,Musik_Asmat,6
493,NaN,NaN,Musik_Asmat,0


In [ ]:
df.columns = ['tgl_akses','id_user','konten','interval']

In [ ]:
#hapus empty value
del_empty = df['konten'] != ''
df  = df[del_empty]

In [ ]:
def clean_outlier(time): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times 
    if re.search('\d{3}', time): 
  
        # Extract the position of beginning of pattern 
        pos = re.search('\d', time.strip('')).start() 
  
        # return the cleaned name 
        return time[:pos] 
  
    else: 
        # if clean up needed return the same name 
        return time 
          
# Updated the city columns 
df['time'] = df['interval'].apply(clean_outlier) 
  
# Print the updated dataframe 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


df['interval'][:15]

In [ ]:
#select with regex
# a = df[df.interval.str.contains('\d{3}', regex= True, na=False)]

del df['interval']

In [ ]:
#delete blank space
df.time = df.time.str.replace(' ', '')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df.time =  df.time.str.lstrip()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df.time =  df.time.str.rstrip()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
# replace '' np nan
new_time = df.time.replace('','15')

In [ ]:
df.time = new_time

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
# df.time.astype(int)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.konten = df.konten.str.replace(' ','')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
#aceh
rumah_aceh   = df['konten'] == 'Rumah_Aceh'
pakaian_aceh = df['konten'] == 'Pakaian_Aceh'
senjata_aceh = df['konten'] == 'Senjata_Aceh'
musik_aceh   = df['konten'] == 'Musik_Aceh'
makanan_aceh = df['konten'] == 'Makanan_Aceh'

#asmat
rumah_asmat   = df['konten'] == 'Rumah_Asmat'
senjata_asmat = df['konten'] == 'Senjata_Asmat'
musik_asmat   = df['konten'] == 'Musik_Asmat'
makanan_asmat = df['konten'] == 'Makanan_Asmat'
pakaian_asmat = df['konten'] == 'Pakaian_Asmat'

#bali
rumah_bali   = df['konten'] == 'Rumah_Bali'
senjata_bali = df['konten'] == 'Senjata_Bali'
musik_bali   = df['konten'] == 'Musik_Bali'
makanan_bali = df['konten'] == 'Makanan_Bali'
pakaian_bali = df['konten'] == 'Pakaian_Bali'

#dayak 
rumah_dayak   = df['konten'] == 'Rumah_Dayak'
senjata_dayak = df['konten'] == 'Senjata_Dayak'
musik_dayak   = df['konten'] == 'Musik_Dayak'
makanan_dayak = df['konten'] == 'Makanan_Dayak'
pakaian_dayak = df['konten'] == 'Pakaian_Dayak'

#sunda
rumah_sunda   = df['konten'] == 'Rumah_Sunda'
senjata_sunda = df['konten'] == 'Senjata_Sunda'
musik_sunda   = df['konten'] == 'Musik_Sunda'
makanan_sunda = df['konten'] == 'Makanan_Sunda'
pakaian_sunda = df['konten'] == 'Pakaian_Sunda'

#toraja
rumah_toraja   = df['konten'] == 'Rumah_Toraja'
senjata_toraja = df['konten'] == 'Senjata_Toraja'
musik_toraja   = df['konten'] == 'Musik_Toraja'
makanan_toraja = df['konten'] == 'Makanan_Toraja'
pakaian_toraja = df['konten'] == 'Pakaian_Toraja'
a = df[pakaian_toraja].time
b = df[pakaian_aceh].time
hasil_a = pd.DataFrame(a)
hasil_b = pd.DataFrame(b)
test_a = np.array(hasil_a)
test_b = np.array(hasil_b)
# test_a.corr(test_b, method='pearson')
# hasil_a.corr(hasil_b,method='pearson')

TypeError: ignored

In [ ]:
import math
temp = ''
temp_tgl = ''

new_df = df.fillna('')

for i,row in new_df.iterrows():
  if row['id_user'] == '':
    row['id_user'] = temp
    row['tgl_akses'] = temp_tgl
  elif temp != row['id_user']:
    temp = row['id_user']
    temp_tgl = row['tgl_akses']

# new_df[50:]

In [ ]:
df.to_excel('/diki.xlsx')